In [117]:
import pandas as pd
from fastai.tabular import * 
import sklearn.metrics as metrics
import numpy as np

In [118]:
df = pd.read_csv('/Users/sharifelfouly/Documents/Data-Mining-Cup/train.csv', sep="|")

In [119]:
df.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 10 columns):
trustLevel                   1879 non-null int64
totalScanTimeInSeconds       1879 non-null int64
grandTotal                   1879 non-null float64
lineItemVoids                1879 non-null int64
scansWithoutRegistration     1879 non-null int64
quantityModifications        1879 non-null int64
scannedLineItemsPerSecond    1879 non-null float64
valuePerSecond               1879 non-null float64
lineItemVoidsPerPosition     1879 non-null float64
fraud                        1879 non-null int64
dtypes: float64(4), int64(6)
memory usage: 146.9 KB


In [122]:
dep = 'fraud'

In [123]:
cat = ['trustLevel']

In [124]:
cont = ['totalScanTimeInSeconds',
        'grandTotal',
        'lineItemVoids',
        'scansWithoutRegistration',
        'quantityModifications',
        'scannedLineItemsPerSecond',
        'valuePerSecond',
        'lineItemVoidsPerPosition'
       ]

In [125]:
procs = [Categorify, Normalize]

In [126]:
valid_idx = range(len(df)-300, len(df))
valid_idx

range(1579, 1879)

In [127]:
data = TabularDataBunch.from_df('.', df, dep, valid_idx=valid_idx, procs=procs, cat_names=cat)
print(data.train_ds.cont_names)
print(data.train_ds.cat_names)

['valuePerSecond', 'totalScanTimeInSeconds', 'scannedLineItemsPerSecond', 'scansWithoutRegistration', 'lineItemVoids', 'lineItemVoidsPerPosition', 'grandTotal', 'quantityModifications']
['trustLevel']


In [169]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(6, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.712082,0.561626,0.843333,00:00
1,0.455287,0.231162,0.916667,00:00
2,0.294477,0.150954,0.940000,00:00
3,0.201717,0.101930,0.953333,00:00
4,0.146185,0.089619,0.966667,00:00
5,0.109790,0.091665,0.963333,00:00


In [159]:
def loss(y_true, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    return (tn * 0 + fp * (-25) + fn * (-5) + tp * 5) / (tn + fp + fn + tp)

In [160]:
y = df['fraud'].iloc[1579:].to_numpy()
y

array([0, 0, 0, 0, ..., 1, 0, 0, 0])

In [170]:
y_hat = [int(learn.predict(df.iloc[i])[1]) for i in range(1579, 1879)]

In [171]:
y_hat = np.array(y_hat)

In [172]:
loss(y_hat, y)

-0.6333333333333333

In [173]:
tn, fp, fn, tp = metrics.confusion_matrix(y, y_hat).ravel()

In [174]:
tp

13

In [175]:
fn

10

In [176]:
fp

1

In [177]:
tn

276

### Accuracy is not bad imo...

In [186]:
acc = (tp+tn)/(tp+fn+fp+tn)
acc

0.9633333333333334